<a href="https://colab.research.google.com/github/keti-Lohith/nlp_projects/blob/main/fake_news_detector_one_hot_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd

In [29]:
df=pd.read_csv("/content/train.csv")

In [30]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [31]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [32]:
df=df.dropna()

In [33]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [60]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [34]:
X=df.drop('label',axis=1)

In [35]:
y=df['label']

In [36]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [37]:
voc_size=5000

In [38]:
messages=X.copy()

In [39]:
messages.reset_index(inplace=True)

In [40]:
import nltk
import re
from nltk.corpus import stopwords

In [41]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [43]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [64]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2862, 4389, 512, 3828, 2698, 4289, 4045, 1235, 2259, 2884],
 [275, 2359, 2812, 1560, 3477, 1577, 1840],
 [4821, 1991, 3223, 1445],
 [1713, 907, 4763, 81, 3832, 665],
 [1496, 3477, 1464, 3393, 973, 647, 3477, 1805, 4836, 1222],
 [2188,
  1564,
  463,
  653,
  4630,
  4106,
  2404,
  128,
  2298,
  4878,
  1646,
  2628,
  2274,
  4989,
  1840],
 [1951, 127, 4040, 4343, 936, 1260, 3442, 4377, 1340, 496, 3376],
 [991, 2922, 4544, 3894, 3148, 3984, 4106, 2509, 1340, 496, 3376],
 [141, 1341, 2385, 4593, 2294, 1559, 3114, 3251, 4106, 2392],
 [4481, 4546, 2628, 891, 3571, 1425, 4571, 1265],
 [58, 1401, 3529, 3304, 3990, 760, 3664, 732, 2990, 720, 3159],
 [81, 2106, 2698, 1559, 4106, 3148],
 [361, 355, 3641, 2991, 250, 2368, 3059, 3102, 1670],
 [4183, 2149, 958, 2574, 3334, 2002, 957, 1340, 496, 3376],
 [2576, 4573, 4343, 3697, 3632, 1340, 496, 3376],
 [599, 1091, 3350, 4842, 2937, 3595, 3035, 3211, 3399, 3445],
 [4998, 3555, 2359],
 [3509, 3507, 419, 2492, 4106, 4822, 3980, 1840],
 [1319, 45

In [46]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2862 4389  512 ...    0    0    0]
 [ 275 2359 2812 ...    0    0    0]
 [4821 1991 3223 ...    0    0    0]
 ...
 [3194 4000 3910 ...    0    0    0]
 [2684 3148 4526 ...    0    0    0]
 [4701 2893 4981 ...    0    0    0]]


In [47]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [57]:
X_final

array([[2862, 4389,  512, ...,    0,    0,    0],
       [ 275, 2359, 2812, ...,    0,    0,    0],
       [4821, 1991, 3223, ...,    0,    0,    0],
       ...,
       [3194, 4000, 3910, ...,    0,    0,    0],
       [2684, 3148, 4526, ...,    0,    0,    0],
       [4701, 2893, 4981, ...,    0,    0,    0]], dtype=int32)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [50]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 38ms/step - loss: 0.2987 - accuracy: 0.8573 - val_loss: 0.2080 - val_accuracy: 0.9094
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1343 - accuracy: 0.9468 - val_loss: 0.2016 - val_accuracy: 0.9201
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0945 - accuracy: 0.9649 - val_loss: 0.2318 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0719 - accuracy: 0.9743 - val_loss: 0.2543 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0505 - accuracy: 0.9839 - val_loss: 0.3252 - val_accuracy: 0.9140
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0356 - accuracy: 0.9887 - val_loss: 0.3564 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0318 - accuracy: 0.9900 - val_loss: 0.3913 - val_accuracy: 0.907

In [51]:
y_pred=model.predict(X_test)

In [52]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [53]:
from sklearn.metrics import confusion_matrix

In [54]:
confusion_matrix(y_test,y_pred)

array([[3160,  259],
       [ 280, 2336]])

In [59]:
X_test

array([[  58, 4408,  333, ...,    0,    0,    0],
       [2551, 4585,  572, ...,    0,    0,    0],
       [1312, 3147, 1215, ...,    0,    0,    0],
       ...,
       [ 292, 4183,  687, ...,    0,    0,    0],
       [4106,    0,    0, ...,    0,    0,    0],
       [3830, 4976, 2499, ...,    0,    0,    0]], dtype=int32)

In [84]:
####testing on users data

In [93]:
review = re.sub('[^a-zA-Z]', ' ', "lottery ticket to get 1 crore")
review = review.lower()
review = review.split() 
review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)


In [94]:
val=one_hot(review,voc_size)

In [95]:
val

[2308, 1048, 3223, 1314]

In [96]:
docs=pad_sequences([val],padding='post',maxlen=sent_length)

In [97]:
x=model.predict(docs)

In [98]:
x

array([[0.9989152]], dtype=float32)

In [99]:
y=np.where(x > 0.6, 1,0)

In [100]:
y

array([[1]])